# Preprocessing

## 1. Imports

### 1.1 Libraries

In [30]:
# builtin
import os, time, sys, random

# data
import pandas as pd
import numpy as np
import requests
import math

# viz
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# NLP
import string
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.corpus import RegexpTokenizer
from nltk.stem.snowball import FrenchStemmer
from collections import Counter
import spacy

# ML
from gensim.models import Word2Vec
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack
import scipy.sparse as sp

# other
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

### 1.2 Download and options

In [31]:
nltk.download('stopwords')
nltk.download('wordpunct')
nltk.download('wordnet')
nltk.download('words')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\derou\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading wordpunct: Package 'wordpunct' not found in
[nltk_data]     index
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\derou\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\derou\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\derou\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [32]:
nlp = spacy.load("fr_core_news_sm")

In [33]:
sns.set()

### 1.3 Loading data

In [34]:
# CSV

#Env Perso
df = pd.read_csv(r"C:\Users\derou\OneDrive\Bureau\DATA\PORTFOLIO\Recommandation de films\CSV\df_movies_cleaned.csv")

# Env Vinci
#df = pd.read_csv(r"C:\Users\melvin.derouk\Desktop\Data formation\Movies-Recommandations\df_movies_cleaned.csv")

## Variable "Synopsis"

### 2. Work on a specific document

In [35]:
# Fonction de saut de ligne pour output
def insert_newlines(string, every=80):
    lines = []
    for i in range(0, len(string), every):
        lines.append(string[i:i+every])
    return '\n'.join(lines)

In [36]:
doc = df.Synopsis.sample(1)
doc = doc.values[0]
print(insert_newlines(doc, every=200))

À Hong Kong, J. S. Cheung est une légende de la police. Un super flic aux états de service impressionnants. Sa spécialité: le déminage, sous toutes ses formes. Lorsque un groupe de terroristes menace 
la ville avec plusieurs tonnes d'explosifs, il est décidé à prendre tous les risques. Mais ces attaques en apparence anarchiques cachent en réalité le casse du siècle.


### 2.1 Lower

In [37]:
doc = doc.lower()

In [38]:
print(insert_newlines(doc, every=200))

à hong kong, j. s. cheung est une légende de la police. un super flic aux états de service impressionnants. sa spécialité: le déminage, sous toutes ses formes. lorsque un groupe de terroristes menace 
la ville avec plusieurs tonnes d'explosifs, il est décidé à prendre tous les risques. mais ces attaques en apparence anarchiques cachent en réalité le casse du siècle.


### 2.2 Tokenization

In [39]:
tokens = word_tokenize(doc)
tokens

['à',
 'hong',
 'kong',
 ',',
 'j.',
 's.',
 'cheung',
 'est',
 'une',
 'légende',
 'de',
 'la',
 'police',
 '.',
 'un',
 'super',
 'flic',
 'aux',
 'états',
 'de',
 'service',
 'impressionnants',
 '.',
 'sa',
 'spécialité',
 ':',
 'le',
 'déminage',
 ',',
 'sous',
 'toutes',
 'ses',
 'formes',
 '.',
 'lorsque',
 'un',
 'groupe',
 'de',
 'terroristes',
 'menace',
 'la',
 'ville',
 'avec',
 'plusieurs',
 'tonnes',
 "d'explosifs",
 ',',
 'il',
 'est',
 'décidé',
 'à',
 'prendre',
 'tous',
 'les',
 'risques',
 '.',
 'mais',
 'ces',
 'attaques',
 'en',
 'apparence',
 'anarchiques',
 'cachent',
 'en',
 'réalité',
 'le',
 'casse',
 'du',
 'siècle',
 '.']

In [40]:
# longueur de la liste
len(tokens)

70

In [41]:
# longueur de la liste (sans les doublons)
len(set(tokens))

56

In [42]:
def display_tokens_infos(tokens):
    """display info about corpus"""

    print(f"nb tokens {len(tokens)}, nb tokens uniques {len(set(tokens))}")
    print(tokens[:30])

In [43]:
tokens = wordpunct_tokenize(doc)
display_tokens_infos(tokens)

nb tokens 74, nb tokens uniques 58
['à', 'hong', 'kong', ',', 'j', '.', 's', '.', 'cheung', 'est', 'une', 'légende', 'de', 'la', 'police', '.', 'un', 'super', 'flic', 'aux', 'états', 'de', 'service', 'impressionnants', '.', 'sa', 'spécialité', ':', 'le', 'déminage']


### 2.3 Stopwords

In [44]:
stop_words = set(stopwords.words('french'))

In [45]:
tokens = [w for w in tokens if w not in stop_words]
display_tokens_infos(tokens)

nb tokens 46, nb tokens uniques 38
['hong', 'kong', ',', '.', '.', 'cheung', 'légende', 'police', '.', 'super', 'flic', 'états', 'service', 'impressionnants', '.', 'spécialité', ':', 'déminage', ',', 'sous', 'toutes', 'formes', '.', 'lorsque', 'groupe', 'terroristes', 'menace', 'ville', 'plusieurs', 'tonnes']


In [46]:
tokenizer = nltk.RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(doc)
display_tokens_infos(tokens)

nb tokens 62, nb tokens uniques 54
['à', 'hong', 'kong', 'j', 's', 'cheung', 'est', 'une', 'légende', 'de', 'la', 'police', 'un', 'super', 'flic', 'aux', 'états', 'de', 'service', 'impressionnants', 'sa', 'spécialité', 'le', 'déminage', 'sous', 'toutes', 'ses', 'formes', 'lorsque', 'un']


In [47]:
tokens = [w for w in tokens if w not in stop_words]
display_tokens_infos(tokens)

nb tokens 34, nb tokens uniques 34
['hong', 'kong', 'cheung', 'légende', 'police', 'super', 'flic', 'états', 'service', 'impressionnants', 'spécialité', 'déminage', 'sous', 'toutes', 'formes', 'lorsque', 'groupe', 'terroristes', 'menace', 'ville', 'plusieurs', 'tonnes', 'explosifs', 'décidé', 'prendre', 'tous', 'risques', 'attaques', 'apparence', 'anarchiques']


### 2.4 First cleaning function

In [48]:
def process_synopsis_1(doc, rejoin=False):

    # lower
    doc = doc.lower().strip()

    # tokenize
    tokenizer = RegexpTokenizer(r'\w+')
    raw_tokens_list = tokenizer.tokenize(doc)

    # stop words
    cleaned_tokens_list = [w for w in raw_tokens_list if w not in stop_words]

    if rejoin : 
        return " ".join(cleaned_tokens_list)
    
    return cleaned_tokens_list

In [49]:
tokens = process_synopsis_1(doc)
display_tokens_infos(tokens)

nb tokens 34, nb tokens uniques 34
['hong', 'kong', 'cheung', 'légende', 'police', 'super', 'flic', 'états', 'service', 'impressionnants', 'spécialité', 'déminage', 'sous', 'toutes', 'formes', 'lorsque', 'groupe', 'terroristes', 'menace', 'ville', 'plusieurs', 'tonnes', 'explosifs', 'décidé', 'prendre', 'tous', 'risques', 'attaques', 'apparence', 'anarchiques']


## 3. Work on the entire corpus

### 3.1 Build raw corpus

In [50]:
raw_corpus = "".join(df.Synopsis.values)
raw_corpus[:3_00]

"Leo, un lézard blasé de 74 ans, vit dans une salle de classe en Floride depuis des décennies, avec une tortue pour copain de terrarium. Aussi, quand il apprend qu'il n'a plus qu'une année à vivre, Leo décide de s'enfuir pour découvrir la vie en liberté. Mais les petits écoliers anxieux le retiennent"

In [51]:
len(raw_corpus)

3831558

In [52]:
corpus = process_synopsis_1(raw_corpus)
display_tokens_infos(corpus)

nb tokens 379423, nb tokens uniques 37830
['leo', 'lézard', 'blasé', '74', 'ans', 'vit', 'salle', 'classe', 'floride', 'depuis', 'décennies', 'tortue', 'copain', 'terrarium', 'aussi', 'quand', 'apprend', 'a', 'plus', 'année', 'vivre', 'leo', 'décide', 'enfuir', 'découvrir', 'vie', 'liberté', 'petits', 'écoliers', 'anxieux']


In [53]:
tmp = pd.Series(corpus).value_counts()
tmp

a                 3849
plus              3258
jeune             2082
vie               1931
alors             1773
                  ... 
vauriens             1
impressionante       1
représentantes       1
corley               1
cacherait            1
Length: 37830, dtype: int64

In [54]:
tmp.head(10)

a        3849
plus     3258
jeune    2082
vie      1931
alors    1773
deux     1692
tout     1642
va       1573
après    1356
faire    1332
dtype: int64

In [55]:
tmp.tail(10)

tobimaru          1
bretteur          1
luan              1
retoucher         1
ashburn           1
vauriens          1
impressionante    1
représentantes    1
corley            1
cacherait         1
dtype: int64

In [57]:
tmp.describe()

count    37830.000000
mean        10.029685
std         53.950711
min          1.000000
25%          1.000000
50%          2.000000
75%          5.000000
max       3849.000000
dtype: float64

### 3.2 List rare tokens

In [58]:
# unique words = usefull ?

tmp = pd.Series(corpus).value_counts()
list_unique_words = tmp[tmp==1]
list_unique_words[:20]

indéterminée     1
israéliennes     1
brûlait          1
guindé           1
aimanter         1
partira          1
jirachi          1
letton           1
enrôla           1
guadeloupe       1
dyslexique       1
empressant       1
srinivasa        1
sondes           1
prolongements    1
behlül           1
trant            1
surfaces         1
épousera         1
barrasser        1
dtype: int64

In [59]:
len(list_unique_words)

16319

In [60]:
list_unique_words = list(list_unique_words.index)
list_unique_words[:20]

['indéterminée',
 'israéliennes',
 'brûlait',
 'guindé',
 'aimanter',
 'partira',
 'jirachi',
 'letton',
 'enrôla',
 'guadeloupe',
 'dyslexique',
 'empressant',
 'srinivasa',
 'sondes',
 'prolongements',
 'behlül',
 'trant',
 'surfaces',
 'épousera',
 'barrasser']

In [61]:
tmp = pd.DataFrame({"words" : list_unique_words})
tmp.to_csv(r"C:\Users\derou\OneDrive\Bureau\DATA\PORTFOLIO\Recommandation de films\CSV\unique_words.csv", index=False)

In [62]:
# idem for min 5 times

tmp = pd.Series(corpus).value_counts()
list_min_5_words = tmp[tmp==5]
list_min_5_words[:20]

apparitions     5
rapporte        5
venture         5
spécialistes    5
révélée         5
défilé          5
charmeur        5
éclaircir       5
suivie          5
joël            5
clémentine      5
options         5
buffalo         5
implique        5
1917            5
stephanie       5
domaines        5
kumail          5
robes           5
assistants      5
dtype: int64

In [63]:
len(list_min_5_words)

1415

In [64]:
list_min_5_words = list(list_min_5_words.index)
list_min_5_words[:20]

['apparitions',
 'rapporte',
 'venture',
 'spécialistes',
 'révélée',
 'défilé',
 'charmeur',
 'éclaircir',
 'suivie',
 'joël',
 'clémentine',
 'options',
 'buffalo',
 'implique',
 '1917',
 'stephanie',
 'domaines',
 'kumail',
 'robes',
 'assistants']

In [65]:
tmp = pd.DataFrame({"words" : list_min_5_words})
tmp.to_csv(r"C:\Users\derou\OneDrive\Bureau\DATA\PORTFOLIO\Recommandation de films\CSV\min_5_words.csv", index=False)

### 3.3 Second cleaning function

In [66]:
def process_synopsis_2(doc,
                       rejoin=False,
                       list_rare_words=None,
                       min_len_word=3,
                       force_is_alpha=True) : 
    
    """cf process_synopsis_1 but with list_unique_words, min_len_word, and force_is_alpha

    positionnal arguments :
    ------------------------
    doc : str : the document (aka a text in str format) to process

    opt args : 
    ------------------------
    rejoin : bool : if True return a string else return the list of tokens
    list_rare_words : list : a list of rare words to exclude
    min_len_word : int : minimum lenght of a word to not exclude
    force_is_alpha : if 1, exclude all tokens with a numeric character

    return : 
    ------------------------
    a string (if rejoin is True) or a list of tokens
    """
    
    # list unique words
    if not list_rare_words:
        list_rare_words = []

    # lower
    doc = doc.lower().strip()

    # tokenize
    tokenizer = RegexpTokenizer(r'\w+')
    raw_tokens_list = tokenizer.tokenize(doc)

    # stop words
    cleaned_tokens_list = [w for w in raw_tokens_list if w not in stop_words]

    #############################################################
    #############################################################

    # no rare tokens
    non_rare_tokens = [w for w in cleaned_tokens_list if w not in list_rare_words]

    # no more len words
    more_than_N = [w for w in non_rare_tokens if len(w) >= min_len_word]

    # only alpha chars
    if force_is_alpha : 
        alpha_tokens = [w for w in more_than_N if w.isalpha()]
    else : 
        alpha_tokens = more_than_N

    #############################################################
    #############################################################

    # manage return type
    if rejoin : 
        return " ".join(alpha_tokens)
    
    return alpha_tokens

In [67]:
display_tokens_infos(corpus)

nb tokens 379423, nb tokens uniques 37830
['leo', 'lézard', 'blasé', '74', 'ans', 'vit', 'salle', 'classe', 'floride', 'depuis', 'décennies', 'tortue', 'copain', 'terrarium', 'aussi', 'quand', 'apprend', 'a', 'plus', 'année', 'vivre', 'leo', 'décide', 'enfuir', 'découvrir', 'vie', 'liberté', 'petits', 'écoliers', 'anxieux']


In [68]:
len(set(corpus))

37830

In [69]:
#3-5 min process

corpus = process_synopsis_2(raw_corpus,
                            list_rare_words=list_unique_words,
                            rejoin=False)
display_tokens_infos(corpus)

nb tokens 350273, nb tokens uniques 21079
['leo', 'lézard', 'blasé', 'ans', 'vit', 'salle', 'classe', 'floride', 'depuis', 'décennies', 'tortue', 'copain', 'aussi', 'quand', 'apprend', 'plus', 'année', 'vivre', 'leo', 'décide', 'enfuir', 'découvrir', 'vie', 'liberté', 'petits', 'écoliers', 'anxieux', 'retiennent', 'craignant', 'notamment']


In [73]:
len(set(corpus))

21079

### 3.4 Stemming & Lemmatize

In [78]:
#test fonction stemming

def process_synopsis_3(doc,
                       rejoin=False,
                       lemm_or_stemm="stem",
                       list_rare_words=None,
                       min_len_word=3,
                       force_is_alpha=True) : 
    
    
    # list unique words
    if not list_rare_words:
        list_rare_words = []

    # lower
    doc = doc.lower().strip()

    # tokenize
    tokenizer = RegexpTokenizer(r'\w+')
    raw_tokens_list = tokenizer.tokenize(doc)

    # stop words
    cleaned_tokens_list = [w for w in raw_tokens_list if w not in stop_words]

    # no rare tokens
    non_rare_tokens = [w for w in cleaned_tokens_list if w not in list_rare_words]

    # no more len words
    more_than_N = [w for w in non_rare_tokens if len(w) >= min_len_word]

    # only alpha chars
    if force_is_alpha : 
        alpha_tokens = [w for w in more_than_N if w.isalpha()]
    else : 
        alpha_tokens = more_than_N

    #############################################################
    #############################################################

    # stem or lem
    if lemm_or_stemm == "lem" : 
        trans = WordNetLemmatizer()
        trans_text = [trans.lemmatize(i) for i in alpha_tokens]
    else : 
        trans = FrenchStemmer()
        trans_text = [trans.stem(i) for i in alpha_tokens]
    #############################################################
    #############################################################

    # manage return type
    if rejoin : 
        return " ".join(trans_text)
    
    return trans_text

In [71]:
#test fonction lemmatize

# def process_synopsis_3(doc,
#                        rejoin=False,
#                        lemm_or_stemm="lem",
#                        list_rare_words=None,
#                        min_len_word=3,
#                        force_is_alpha=True) : 
    
    
#     # list unique words
#     if not list_rare_words:
#         list_rare_words = []

#     # lower
#     doc = doc.lower().strip()

#     # tokenize
#     tokenizer = RegexpTokenizer(r'\w+')
#     raw_tokens_list = tokenizer.tokenize(doc)

#     # stop words
#     cleaned_tokens_list = [w for w in raw_tokens_list if w not in stop_words]

#     # no rare tokens
#     non_rare_tokens = [w for w in cleaned_tokens_list if w not in list_rare_words]

#     # no more len words
#     more_than_N = [w for w in non_rare_tokens if len(w) >= min_len_word]

#     # only alpha chars
#     if force_is_alpha : 
#         alpha_tokens = [w for w in more_than_N if w.isalpha()]
#     else : 
#         alpha_tokens = more_than_N

#     #############################################################
#     #############################################################

#     # stem or lem
#     if lemm_or_stemm == "lem" : 
#         trans = nlp(' '.join(alpha_tokens))
#         trans_text = [trans.lemma_(i) for i in alpha_tokens]
#     else : 
#         trans = FrenchStemmer()
#         trans_text = [trans.stem(i) for i in alpha_tokens]
#     #############################################################
#     #############################################################

#     # manage return type
#     if rejoin : 
#         return " ".join(trans_text)
    
#     return trans_text

In [79]:
corpus = process_synopsis_3(raw_corpus, rejoin=False, list_rare_words=list_unique_words)
pd.Series(corpus).sample(20)

247808    créatur
226702     garçon
297288     empêch
95703      troubl
162101    austral
199540       sien
340466        lan
164011      coffr
73567      biolog
110007     chacun
222131     combat
330460     volont
195766        art
119228       tout
232069    instabl
76860         peu
307485    victoir
330031    peuvent
192519        peu
223030     affect
dtype: object

In [ ]:
len(set(corpus))

12874

### 3.5 Wordcloud

In [83]:
wordcloud = WordCloud(width = 800,
                      height = 400,
                      background_color='white',
                      stopwords=[],
                      max_words=50).generate(" ".join(corpus))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

ValueError: Only supported for TrueType fonts

## 4. Final cleaning function

In [84]:
def final_clean(doc) :

    new_doc = process_synopsis_3(doc,
                                 rejoin=False,
                                 lemm_or_stemm="stem",
                                 list_rare_words=list_unique_words,
                                 min_len_word=3,
                                 force_is_alpha=True)
    return new_doc

In [85]:
df['clean_synopsis'] = df.Synopsis.apply(final_clean)

In [87]:
# on reconverti la colonne clean_synopsis_str en chaine de char
df['clean_synopsis_str'] = df['clean_synopsis'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

In [88]:
df.head(5)

,ID,Titre,Date de sortie,Genre,Note,Popularité,Synopsis,Affiche,clean_synopsis,clean_synopsis_str
0,1075794,Leo,2023-11-17,"['Animation', 'Comédie', 'Familial']",7.7,1828.905,"Leo, un lézard blasé de 74 ans, vit dans une s...",/pD6sL4vntUOXHmuvJPPZAgvyfd9.jpg,"[leo, lézard, blas, an, vit, sall, class, flor...",leo lézard blas an vit sall class florid depui...
1,872585,Oppenheimer,2023-07-19,"['Drame', 'Histoire']",8.2,1142.284,"En 1942, convaincus que l'Allemagne nazie est ...",/boAUuJBeID7VNp4L7LNMQs8mfQS.jpg,"[convaincus, allemagn, naz, train, développ, a...",convaincus allemagn naz train développ arme nu...
2,901362,Les Trolls 3,2023-10-12,"['Animation', 'Familial', 'Musique', 'Fantasti...",7.2,1121.642,Après deux films à se tourner autour pour fina...,/r5uiTUeqoERL66fNgtPVbTViHml.jpg,"[apres, deux, film, tourn, autour, final, tomb...",apres deux film tourn autour final tomb bras a...
3,507089,Five Nights at Freddy's,2023-10-25,"['Horreur', 'Mystère']",7.9,779.775,"Mike, jeune homme perturbé, s’occupe de sa sœu...",/tEY81I7lpiHaLJa7AZ3O4vWXmJo.jpg,"[mik, jeun, homm, perturb, occup, sœur, abby, ...",mik jeun homm perturb occup sœur abby âgé an t...
4,670292,The Creator,2023-09-27,"['Science-Fiction', 'Action', 'Thriller']",7.1,774.741,Alors qu'une future guerre entre la race humai...,/pP1cyoXFc5Br1Sg21uORSN49yyu.jpg,"[alor, futur, guerr, entre, rac, humain, intel...",alor futur guerr entre rac humain intelligent ...


## 5. Date de sortie to datetime

In [90]:
df['Date de sortie'] = pd.to_datetime(df['Date de sortie'])

### 5.1 Age of movies

In [91]:
current_year = datetime.now().year
df['Age du film'] = current_year - df['Date de sortie'].dt.year

## 6. Binary encoding of genres

In [111]:
print(len(df))
print(df['Genre'].apply(lambda x: isinstance(x, str)).sum())

9182
9182


In [112]:
df['Genre'] = df['Genre'].apply(lambda x: eval(x) if isinstance(x, str) else x)
print(df['Genre'].apply(lambda x: isinstance(x, list)).sum())

9182


In [113]:
mlb = MultiLabelBinarizer()
genre_binarized = mlb.fit_transform(df['Genre'])

genre_df = pd.DataFrame(genre_binarized, columns=mlb.classes_)

In [114]:
genre_df.index = df.index

In [115]:
df = pd.concat([df.drop('Genre', axis=1), genre_df], axis=1)

# 7. Final df

In [92]:
# df_final = df.copy()

In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9182 entries, 0 to 9181
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ID                  9182 non-null   int64         
 1   Titre               9182 non-null   object        
 2   Date de sortie      9182 non-null   datetime64[ns]
 3   Note                9182 non-null   float64       
 4   Popularité          9182 non-null   float64       
 5   Synopsis            9182 non-null   object        
 6   Affiche             9182 non-null   object        
 7   clean_synopsis      9182 non-null   object        
 8   clean_synopsis_str  9182 non-null   object        
 9   Age du film         9182 non-null   int64         
 10  Action              9182 non-null   int32         
 11  Animation           9182 non-null   int32         
 12  Aventure            9182 non-null   int32         
 13  Comédie             9182 non-null   int32       

In [94]:
# df_final['tags'] = df_final['clean_synopsis_str']+df_final['Genre']

In [102]:
# del_columns = ['Genre', 'clean_synopsis', 'clean_synopsis_str']

# df_final.drop(columns=del_columns, inplace=True)

In [101]:
# df_final.columns

Index(['ID', 'Titre', 'Date de sortie', 'Genre', 'Note', 'Popularité',
       'Synopsis', 'Affiche', 'clean_synopsis', 'clean_synopsis_str',
       'Age du film', 'tags'],
      dtype='object')

In [106]:
# df_final = df_final[['ID', 'Titre', 'Date de sortie', 'Age du film', 'Note', 'Popularité',
#                      'Synopsis', 'tags', 'Affiche']]

In [117]:
df.to_csv(r"C:\Users\derou\OneDrive\Bureau\DATA\PORTFOLIO\Recommandation de films\CSV\df_movies_preprocess.csv", index=False)
genre_df.to_csv(r"C:\Users\derou\OneDrive\Bureau\DATA\PORTFOLIO\Recommandation de films\CSV\genres_binarized.csv", index=False)